In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt

In [2]:
image_path = '/content/drive/My Drive/project/AOI/data/train_images/train_images/'
# image_path = '/content/drive/My Drive/project/AOI/data/test_images/test_images/'
csv_path = '/content/drive/My Drive/project/AOI/data/'

In [ ]:
# df = pd.read_csv(os.path.join(csv_path, 'train.csv'))
df = pd.read_csv(os.path.join(csv_path, 'test.csv'))
n_a = np.array(df)

# Write TFRecord Dataset

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list = tf.train.FloatList(value = [value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array


# def image_example(image, category):
#     # image = plt.imread(os.path.join(image_path, file_name))
#     # image_string = open(os.path.join(image_path, file_name), 'rb').read()
#     # image_shape = tf.image.decode_jpeg(image_string).shape
#     image_string = tf.image.encode_jpeg(image)
#     feature = {
#       'target':_bytes_feature(category),
#       'image_raw': _bytes_feature(image_string),
#     }
#     return tf.train.Example(features = tf.train.Features(feature = feature))

def image_example(image):
  image_string = tf.image.encode_jpeg(image)
  feature = {
      'image_raw' : _bytes_feature(image_string)
  }
  return tf.train.Example(features = tf.train.Features(feature = feature))

def one_hot(label):
  o_h_e = np.zeros((1,6), dtype = 'float32')
  if label == 0:
    o_h_e[0,0] = 1
  elif label == 1:
    o_h_e[0,1] = 1
  elif label == 2:
    o_h_e[0,2] = 1
  elif label == 3:
    o_h_e[0,3] = 1
  elif label == 4:
    o_h_e[0,4] = 1
  else:
    o_h_e[0, 5] = 1
  # return o_h_e.tostring()
  return o_h_e.tobytes()

In [ ]:
image_gen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 15,            # 隨機旋轉 ±45°
    width_shift_range = .15,        # 隨機水平移動 ±15%
    height_shift_range = .15,       # 隨機垂直移動 ±15%
    horizontal_flip = True,         # 隨機水平翻轉
    zoom_range = 0.2,
)

In [ ]:
j

19278

In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'test.tfrecord')) as writer:
  j = 0
  for img_name in n_a[:, 0]:
    if os.path.isfile(image_path + img_name):
      img = plt.imread(os.path.join(image_path, img_name))
      img = cv2.resize(img, (224, 224))
      tf_example = image_example(img)
      writer.write(tf_example.SerializeToString())
      j += 1
writer.close()
print(j)

19278


In [ ]:
with tf.io.TFRecordWriter(os.path.join(csv_path, 'train.tfrecord')) as writer:
  j = 0
  for img_name in n_a[:, 0]:
    i = 0
    if os.path.isfile(image_path + img_name):
      img = plt.imread(os.path.join(image_path, img_name))
      img = np.expand_dims(img, axis = 0)
      category = one_hot(n_a[j][1])
      for batch in image_gen_train.flow(img, batch_size = 32):
        augImage = batch[0]
        augImage = augImage.astype('float32')
        augImage = cv2.resize(augImage, (224, 224))
        tf_example = image_example(augImage, category)
        writer.write(tf_example.SerializeToString())
        i += 1
        if i >= 20:
          break
    j += 1
writer.close()
print(j)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


5056


# Read TFRecord Dataset

In [31]:
BATCH_SIZE = 512
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [34]:
def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    # image = image/255
    # image = tf.reshape(image, [224, 224, 3])
    return image

def decode_label(label):
  label = tf.io.decode_raw(label, tf.float32)
  # label = tf.cast(label, tf.float32)
  return label

def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.string)
        }
        if labeled
        else {"image_raw": tf.io.FixedLenFeature([], tf.string)}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image_raw'])
    if labeled:
        label = decode_label(example['target'], )
        # label = tf.cast(label, tf.int32)
        return image, label
    return image

def load_dataset(filenames, labeled = False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = True  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  
# uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled = labeled), num_parallel_calls = AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, BATCH_SIZE, labeled = False):
    dataset = load_dataset(filenames, labeled = labeled)
#     dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size = AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [38]:
train_dataset = get_dataset(os.path.join(csv_path, 'train.tfrecord'), 1024, labeled = True)

In [39]:
image_batch, label_batch = next(iter(train_dataset))

In [40]:
np.unique(label_batch, axis = 0)

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
def show_batch(image_batch):
  plt.figure(figsize = (20, 10))
  for n in range(200):
    ax = plt.subplot(20, 10, n + 1)
    plt.imshow(image_batch[n] / 255.0)

    plt.axis('off')

show_batch(image_batch.numpy())